In [64]:
import requests
import json
import csv
import re
import pandas as pd
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

In [2]:
measure_file = "measurecsv.csv"
measure_name = []
expression_code = []
dax_url = []

In [3]:
data =pd.read_csv("measurecsv.csv")
df = pd.DataFrame(data)
df

,Measure,expression
0,Availability,\nvar MaxDate = MAX ( 'Date Dimension'[Date] )...
1,Average Daily Occupancy,\nAVERAGEX( \n VALUES('Date Dimension'[Date...
2,Average Daily Occupied Units,\nAVERAGEX( \n VALUES('Date Dimension'[Date...
3,Average Length of Stay,\nvar MaxDateKey = MAX ( 'Date Dimension'[Date...
4,Cancelled Housing Contracts,\nvar CancelledHousingContracts = \nCALCULATET...
5,Unit Occupancy %,"\nDIVIDE([Occupied Units], [Total Units], 0)"
6,Individual Move Ins,\nvar MoveIns =\nCALCULATETABLE(\n 'Housing...
7,Individual Move Outs,\nvar MoveOuts =\nCALCULATETABLE(\n 'Housin...
8,Occupied Units,\nvar MaxDateKey = MAX ( 'Date Selection'[Date...
9,Residents,\nvar MaxDateKey = MAX('Date Dimension'[Date])...


In [72]:
df['expression'] = df['expression'].apply(lambda x:x.strip().replace("\n"," "))
df

,Measure,expression,measureExp,daxurl
0,Availability,var MaxDate = MAX ( 'Date Dimension'[Date] )va...,Availability = var MaxDate = MAX ( 'Date Dimen...,http://www.daxformatter.com/?fx=Availability =...
1,Average Daily Occupancy,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupancy = AVERAGEX( VALUES...,http://www.daxformatter.com/?fx=Average Daily ...
2,Average Daily Occupied Units,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupied Units = AVERAGEX( V...,http://www.daxformatter.com/?fx=Average Daily ...
3,Average Length of Stay,var MaxDateKey = MAX ( 'Date Dimension'[Date] ...,Average Length of Stay = var MaxDateKey = MAX ...,http://www.daxformatter.com/?fx=Average Length...
4,Cancelled Housing Contracts,var CancelledHousingContracts = CALCULATETABLE...,Cancelled Housing Contracts = var CancelledHou...,http://www.daxformatter.com/?fx=Cancelled Hous...
5,Unit Occupancy %,"DIVIDE([Occupied Units], [Total Units], 0)","Unit Occupancy % = DIVIDE([Occupied Units], [T...",http://www.daxformatter.com/?fx=Unit Occupancy...
6,Individual Move Ins,var MoveIns =CALCULATETABLE( 'Housing Contr...,Individual Move Ins = var MoveIns =CALCULATETA...,http://www.daxformatter.com/?fx=Individual Mov...
7,Individual Move Outs,var MoveOuts =CALCULATETABLE( 'Housing Cont...,Individual Move Outs = var MoveOuts =CALCULATE...,http://www.daxformatter.com/?fx=Individual Mov...
8,Occupied Units,var MaxDateKey = MAX ( 'Date Selection'[Date] ...,Occupied Units = var MaxDateKey = MAX ( 'Date ...,http://www.daxformatter.com/?fx=Occupied Units...
9,Residents,var MaxDateKey = MAX('Date Dimension'[Date])va...,Residents = var MaxDateKey = MAX('Date Dimensi...,http://www.daxformatter.com/?fx=Residents = va...


In [73]:
df["measureExp"] = df["Measure"].map(str) + " = " + df["expression"]
df

,Measure,expression,measureExp,daxurl
0,Availability,var MaxDate = MAX ( 'Date Dimension'[Date] )va...,Availability = var MaxDate = MAX ( 'Date Dimen...,http://www.daxformatter.com/?fx=Availability =...
1,Average Daily Occupancy,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupancy = AVERAGEX( VALUES...,http://www.daxformatter.com/?fx=Average Daily ...
2,Average Daily Occupied Units,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupied Units = AVERAGEX( V...,http://www.daxformatter.com/?fx=Average Daily ...
3,Average Length of Stay,var MaxDateKey = MAX ( 'Date Dimension'[Date] ...,Average Length of Stay = var MaxDateKey = MAX ...,http://www.daxformatter.com/?fx=Average Length...
4,Cancelled Housing Contracts,var CancelledHousingContracts = CALCULATETABLE...,Cancelled Housing Contracts = var CancelledHou...,http://www.daxformatter.com/?fx=Cancelled Hous...
5,Unit Occupancy %,"DIVIDE([Occupied Units], [Total Units], 0)","Unit Occupancy % = DIVIDE([Occupied Units], [T...",http://www.daxformatter.com/?fx=Unit Occupancy...
6,Individual Move Ins,var MoveIns =CALCULATETABLE( 'Housing Contr...,Individual Move Ins = var MoveIns =CALCULATETA...,http://www.daxformatter.com/?fx=Individual Mov...
7,Individual Move Outs,var MoveOuts =CALCULATETABLE( 'Housing Cont...,Individual Move Outs = var MoveOuts =CALCULATE...,http://www.daxformatter.com/?fx=Individual Mov...
8,Occupied Units,var MaxDateKey = MAX ( 'Date Selection'[Date] ...,Occupied Units = var MaxDateKey = MAX ( 'Date ...,http://www.daxformatter.com/?fx=Occupied Units...
9,Residents,var MaxDateKey = MAX('Date Dimension'[Date])va...,Residents = var MaxDateKey = MAX('Date Dimensi...,http://www.daxformatter.com/?fx=Residents = va...


In [74]:
df['daxurl'] = "http://www.daxformatter.com/?fx=" + df['measureExp'].map(str) + "&r=US"
df

,Measure,expression,measureExp,daxurl
0,Availability,var MaxDate = MAX ( 'Date Dimension'[Date] )va...,Availability = var MaxDate = MAX ( 'Date Dimen...,http://www.daxformatter.com/?fx=Availability =...
1,Average Daily Occupancy,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupancy = AVERAGEX( VALUES...,http://www.daxformatter.com/?fx=Average Daily ...
2,Average Daily Occupied Units,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupied Units = AVERAGEX( V...,http://www.daxformatter.com/?fx=Average Daily ...
3,Average Length of Stay,var MaxDateKey = MAX ( 'Date Dimension'[Date] ...,Average Length of Stay = var MaxDateKey = MAX ...,http://www.daxformatter.com/?fx=Average Length...
4,Cancelled Housing Contracts,var CancelledHousingContracts = CALCULATETABLE...,Cancelled Housing Contracts = var CancelledHou...,http://www.daxformatter.com/?fx=Cancelled Hous...
5,Unit Occupancy %,"DIVIDE([Occupied Units], [Total Units], 0)","Unit Occupancy % = DIVIDE([Occupied Units], [T...",http://www.daxformatter.com/?fx=Unit Occupancy...
6,Individual Move Ins,var MoveIns =CALCULATETABLE( 'Housing Contr...,Individual Move Ins = var MoveIns =CALCULATETA...,http://www.daxformatter.com/?fx=Individual Mov...
7,Individual Move Outs,var MoveOuts =CALCULATETABLE( 'Housing Cont...,Individual Move Outs = var MoveOuts =CALCULATE...,http://www.daxformatter.com/?fx=Individual Mov...
8,Occupied Units,var MaxDateKey = MAX ( 'Date Selection'[Date] ...,Occupied Units = var MaxDateKey = MAX ( 'Date ...,http://www.daxformatter.com/?fx=Occupied Units...
9,Residents,var MaxDateKey = MAX('Date Dimension'[Date])va...,Residents = var MaxDateKey = MAX('Date Dimensi...,http://www.daxformatter.com/?fx=Residents = va...


In [61]:
df_small = df[0:4]
df_small

,Measure,expression,measureExp,daxurl
0,Availability,var MaxDate = MAX ( 'Date Dimension'[Date] )va...,Availability = var MaxDate = MAX ( 'Date Dimen...,http://www.daxformatter.com/?fx=Availability =...
1,Average Daily Occupancy,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupancy = AVERAGEX( VALUES...,http://www.daxformatter.com/?fx=Average Daily ...
2,Average Daily Occupied Units,"AVERAGEX( VALUES('Date Dimension'[Date]), ...",Average Daily Occupied Units = AVERAGEX( V...,http://www.daxformatter.com/?fx=Average Daily ...
3,Average Length of Stay,var MaxDateKey = MAX ( 'Date Dimension'[Date] ...,Average Length of Stay = var MaxDateKey = MAX ...,http://www.daxformatter.com/?fx=Average Length...


In [70]:
print(df.iloc[0,3])

http://www.daxformatter.com/?fx=Availability = var MaxDate = MAX ( 'Date Dimension'[Date] )var AllUnits =CALCULATETABLE(    VALUES('Unit Historical Facts'[UnitKey]),    'Unit Dimension'[Unit Start Effective DateTime] <= MaxDate,    'Unit Dimension'[Unit End Effective DateTime] >= MaxDate ||    ISBLANK ( 'Unit Dimension'[Unit End Effective DateTime] ) ,    'Unit Dimension'[CountTowardsTotal] = 1)var OccupiedUnits = CALCULATETABLE(    VALUES('Housing Contract Facts'[UnitKey]),    CROSSFILTER('Housing Contract Facts'[MoveInDate], 'Date Dimension'[Date], None),    'Housing Contract Facts'[MoveInDate] <= MaxDate,    'Housing Contract Facts'[MoveOutDate] >= MaxDate ||    ISBLANK ( 'Housing Contract Facts'[MoveOutDate] ),        ISBLANK ( 'Housing Contract Facts'[CancelledDate] ) ,    'Unit Dimension'[CountTowardsTotal] = 1)var VacantUnits = EXCEPT(AllUnits ,OccupiedUnits)                                          var PendingMoveOuts = CALCULATETABLE(    VALUES('Housing Contract Facts'[UnitKey

In [75]:
browser = webdriver.Chrome() 

# Make request and store response
# response = requests.get(url)
failed = []
success = []

for i in range(len(df)) : 
    try:
        browser.get(df.iloc[i,3]) #navigate to the page
        time.sleep(5)
        browser.find_element_by_xpath('//*[@id="header"]/div/div[11]/a[4]').click() #save as docx
        time.sleep(2)
        success.append(df.iloc[i,0])
        time.sleep(2)

    except:
        failed.append(df.iloc[i,0])
        pass


In [59]:
browser = webdriver.Chrome()
browser.get(df.iloc[1,3]) #navigate to the page
browser.implicitly_wait(5)
browser.find_element_by_xpath('//*[@id="header"]/div/div[11]/a[4]').click()

In [76]:
print((success))
print ((failed))


['Average Daily Occupancy', 'Average Daily Occupied Units', 'Cancelled Housing Contracts', 'Unit Occupancy %', 'Individual Move Ins', 'Individual Move Outs', 'Occupied Units', 'Stays', 'Transfer Ins', 'Transfer Outs', 'Unit Move Ins', 'Unit Move Outs', 'Vacant Units', 'Tour to Move In', 'Inquiry to Move In', 'Avg Days Inquiry to Move-In', 'Inquiries per Move In', 'Tours per Move In', 'Call Outs per Move In', 'Created Housing Contracts', 'Occupancy %', 'Occupancy', 'Birthday Individuals', 'Total Units', 'Capacity', 'Attended', 'Invited', 'Attendance Rate', 'Avg Attendance Rate', 'Avg Campaigns Attended by Move Ins', 'Total Campaigns Attended', 'Campaign Budget', 'Cost per Attendee', 'Campaign Individuals', 'Contacts With Missing Fields', 'Transactions', 'Floorplan Rank', 'Waitlist Individuals', 'Waitlist Rank', 'All Completed Activities', 'Avg Days Inquiry to Move-In (Activity)', 'Deposit to Move In (Activities)', 'Deposits', 'Initial Deposits', 'Initial Move-In (Activities)', 'Initial 

In [ ]:
df.iloc[0,2]

In [10]:
# Specify the URL
url = df.iloc[4,3]


# Make request and store response
response = requests.get(url)

# Print status code
print(response.status_code)


200


In [12]:
browser = webdriver.Chrome() 
browser.get(df.iloc[4,3]) #navigate to the page
browser.find_element_by_xpath('//*[@id="header"]/div/div[5]/a[3]').click() #save as docx

In [ ]:
def get_formatted_dax(dax_url):
    try:
        temp_data = requests.get(dax_url).json()['data']
        return pd.DataFrame(temp_data)
    except Exception as e:
            print(str(dax_url, str(e)))

frames = []

for page in pages_to_get:
    frames.append(get_formatted_dax(page))

In [6]:
with open(measure_file, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        measure = (row["Measure"])
        measure_name.append(measure)
        expression = (row["expression"])
        expression_code.append(expression)

In [9]:
combine = dict(zip(measure_name, expression_code))
for m,e in combine.items():
    url = "http://www.daxformatter.com/?fx={0}={1}&r=US".format(m,e)
#     url = re.sub("s\s+", " ", url)
    url = url.replace("\n", "")
    url = url.replace(" ", "%20")
    dax_url.append(url)
dax_url

["http://www.daxformatter.com/?fx=Availability=\nvar%20MaxDate%20=%20MAX%20(%20'Date%20Dimension'[Date]%20)\n\nvar%20AllUnit%20=\nCALCULATETABLE(\n%20%20%20%20VALUES('Unit%20Historical%20Facts'[UnitKey]),\n%20%20%20%20'Unit%20Dimension'[Unit%20Start%20Effective%20DateTime]%20<=%20MaxDate,\n%20%20%20%20'Unit%20Dimension'[Unit%20End%20Effective%20DateTime]%20>=%20MaxDate%20||\n%20%20%20%20ISBLANK%20(%20'Unit%20Dimension'[Unit%20End%20Effective%20DateTime]%20)%20,\n%20%20%20%20'Unit%20Dimension'[CountTowardsTotal]%20=%201\n)\n\nvar%20OccupiedUnit%20=%20\nCALCULATETABLE(\n%20%20%20%20VALUES('Housing%20Contract%20Facts'[UnitKey]),\n%20%20%20%20CROSSFILTER('Housing%20Contract%20Facts'[MoveInDate],%20'Date%20Dimension'[Date],%20None),\n%20%20%20%20'Housing%20Contract%20Facts'[MoveInDate]%20<=%20MaxDate,\n%20%20%20%20'Housing%20Contract%20Facts'[MoveOutDate]%20>=%20MaxDate%20||\n%20%20%20%20ISBLANK%20(%20'Housing%20Contract%20Facts'[MoveOutDate]%20),%20%20%20%20\n%20%20%20%20ISBLANK%20(%20'Hou

In [45]:
# URL for GET requests to retrieve vehicle data
url_root = "http://www.daxformatter.com/?fx={}&r=US"

#to pull multiple pages
pages_to_get = [url_root.format(x) for x in range(1,50)]

In [ ]:
#alternative
#reading json to pandas df

url = http....
df = pd.read_json(url, orient='columns')

In [37]:
# Print the response object to the console
response = requests.get(url).json()


In [38]:
#find keys in data
response.keys()

#alt
#data = json_data['key1'].get('desired key')[0].keys()

dict_keys(['is_authenticated', 'page_count', 'average_rating', 'authentication_timeout', 'user', 'total', 'data', 'page'])

In [39]:
review_data = response.get('data') # returns a list
review_data

[{'author_age': '47',
  'author_city': 'Sammamish',
  'author_name': 'Amy',
  'author_state': 'WA',
  'id': '113927310',
  'is_verified_purchaser': False,
  'product_slug': 'bike',
  'rating': 5,
  'rating_quality': 5,
  'rating_value': 5,
  'review_text': 'My husband purchased the bike about a year ago. I first thought another work out equipment that we will never use. Well I was wrong! After having our kids I stopped working out, as I could never find the time. Well the bike changed all that,  I love getting on the bike and taking the time for me. Our kids know now when mom is  on the bike it’s her time. They also love the bike and want to ride like Mom and Dad. I purchased the clips so they can now ride and the love it. \nI ‘m now running my first half marathon  in June.',
  'ride_because': 'ToImproveFitness',
  'ride_frequency': 'AFewTimesAWeek',
  'score': 10,
  'share': 'IShareWithFamily',
  'submitted_at': 1527261091,
  'support_comments': [],
  'title': 'Love , love  this bike.

In [42]:
#use pandas to make it look nice
pd.DataFrame(review_data)[:5]

,author_age,author_city,author_name,author_state,id,is_verified_purchaser,product_slug,rating,rating_quality,rating_value,review_text,ride_because,ride_frequency,score,share,submitted_at,support_comments,title,year_joined
0,47,Sammamish,Amy,WA,113927310,False,bike,5,5.0,5.0,My husband purchased the bike about a year ago...,ToImproveFitness,AFewTimesAWeek,10,IShareWithFamily,1527261091,[],"Love , love this bike. Life changing!!!",2017
1,34,Fresno,Arnette Dunn,CA,113889277,True,bike,5,5.0,5.0,I bought this bike 1 year ago and I must say I...,ToLoseWeight,AFewTimesAWeek,10,JustMe,1527133493,"[{'department': 'Team Peloton', 'comment_text'...",Best decision I've ever made!,2017
2,47,Barrington,Lisa,RI,113880113,True,bike,5,4.0,3.0,"I bought this as a gift for my husband, so he ...",ToImproveFitness,AFewTimesAWeek,10,IShareWithASignificantOther,1527103838,[],I love my husband's Christmas gift!,2017
3,53,Ridgecrest,Staci B,CA,113498295,True,bike,5,5.0,5.0,After wanting one of these bikes for almost 2 ...,ToLoseWeight,EveryDay,10,JustMe,1526836693,"[{'department': 'Team Peloton', 'comment_text'...",Finally! A Piece of Equipment I LOVE!,2017
4,None,Highlands Ranch,Jon Nordmark,CO,113440215,False,bike,5,5.0,5.0,My wife and I bought a Peloton 2.5 years ago (...,ToImproveFitness,AFewTimesAWeek,10,IShareWithASignificantOther,1526653957,[],Tremendous Fitness Experience,2015


In [51]:
def get_api_data(some_url):
    try:
        temp_data = requests.get(some_url).json()['data']
        return pd.DataFrame(temp_data)
    except Exception as e:
            print(str(some_url, str(e)))

frames = []

for page in pages_to_get:
    frames.append(get_api_data(page))

In [54]:
df = pd.concat(frames)

In [57]:
detractors = df.query('rating < 4').copy()
detractors[:5]

,author_age,author_city,author_name,author_state,id,is_verified_purchaser,product_slug,rating,rating_quality,rating_value,review_text,ride_because,ride_frequency,score,share,submitted_at,support_comments,title,year_joined
5,30,Brentwood,Marissa Dyer,CA,113381979,True,bike,2,NaN,NaN,I recommended the Pelaton for my mother after ...,None,None,2,None,1526402185,"[{'department': 'Team Peloton', 'comment_text'...",Delivery is a mess,None
0,36,None,Ryan McDonnell,None,113173122,True,bike,3,NaN,NaN,see anything else section. I’m hoping the is...,None,None,5,None,1525651598,"[{'department': 'Team Peloton', 'comment_text'...",Bike issues,None
3,21,Surat,Axay Bhathela,IN,113044682,False,bike,3,5.0,5.0,Took my first class today. Fantastic experienc...,ToLoseWeight,AFewTimesAMonth,10,JustMe,1525251563,[],MY VERY OWN PERSONAL TRAINING!!!,2014
1,57,New brunswick,Tom,NJ,112715970,True,bike,3,2.0,3.0,After 30+ rides I am finding streaming lockups...,ToImproveFitness,AFewTimesAWeek,4,JustMe,1524615861,"[{'department': 'Team Peloton', 'comment_text'...","Bike is great, streaming is horrible.",2017
8,None,None,Mike,None,112212006,True,bike,1,1.0,1.0,If it was possible to give this product a nega...,ToLoseWeight,AFewTimesAWeek,1,IShareWithASignificantOther,1523895035,"[{'department': 'Team Peloton', 'comment_text'...",complete waste of time and money,2017
